In [1]:
import pandas as pd
import numpy as np 
from scipy.stats import norm 
import scipy.interpolate

hist_rates = pd.read_csv('spot_rates.csv')
hist_rates.columns = ['date', 'JPY', 'USD', 'GBP', 'MYR', 'CHF', 'EUR', 'NZD']

hist_rates_ret = pd.DataFrame()

currencies = ['EUR', 'USD', 'JPY', 'CHF']

for x in currencies:
    hist_rates_ret[x] = np.log(hist_rates[x]).diff()
    
HS_sim_price = hist_rates_ret * [0.6148, 0.6812, 72.97, 0.6738]
HS_sim_price = HS_sim_price + [0.6148, 0.6812, 72.97, 0.6738]
HS_sim_price = (HS_sim_price.drop(HS_sim_price.index[0]))

#Bond_portfolio.to_excel (r'C:\Users\Mat\Documents\bonds.xlsx', index = None, header=True)
HS_sim_price.to_excel (r'C:\Users\Mat\Documents\HS_sim_price.xlsx', index = None, header=True)

In [2]:
##For options 
opt_currencies = ['EUR', 'USD', 'JPY']
HS_price_opt = pd.DataFrame() 
for x in opt_currencies:
    HS_price_opt[x] = HS_sim_price[x]
    
#insert cloned colums for EUR and USD 
HS_price_opt.insert(1, 'EUR2', HS_price_opt['EUR'])
HS_price_opt.insert(3, 'USD2', HS_price_opt['USD'])



In [3]:
#Calc r, rf, and vol for options 
#to interp rates for options
X = [0,30.41666667,60.83333333,91.25,182.5,273.75,365,730,1095,1460,1825]
Aus = [1.0449, 1.0344, 1.0138, 1.0335, 0.9978, 0.9295, 0.9044, 0.7954, 0.8295, 0.9346, 0.9839]
EUR = [-0.4132,-0.3923,-0.3919,-0.3765,-0.3627,-0.436,-0.4141,-0.4451,-0.4346,-0.4014,-0.3522]
USD = [2.1265,2.2513,2.2667,2.258,2.1328,2.0367,1.9437,1.7289,1.6493,1.6212,1.6207]
JPY = [-0.0824,-0.0991,-0.094,-0.0723,-0.03583,-0.0384,-0.0493,-0.0801,-0.0924,-0.0949,-0.09]
XCHF = [0,30.41666667,91.25,182.5,273.75,365,730,1095,1825]
CHF = [-0.7868,-0.7849,-0.7066,-0.8154,-0.8224,-0.8549,-0.8528,-0.8458,-0.7577]

options = pd.read_csv('Curr_opt.csv')

#Inteerpolate AUS rates for given maturities 
mats = (315, 274, 61, 122, 187)
rforo = []
for x in mats:
    interp = scipy.interpolate.interp1d(X, Aus, bounds_error=False, fill_value=scipy.nan)
    rforo.append(float(interp(x)))  
options['r'] = rforo    

#interpolate RF for given currencies 
#Euro interp 
interp = scipy.interpolate.interp1d(X, EUR, bounds_error=False, fill_value=scipy.nan)
options.iloc[0,10] = interp(315)
options.iloc[1,10] = interp(274)

#USD interp
interpu = scipy.interpolate.interp1d(X, USD, bounds_error=False, fill_value=scipy.nan)
options.iloc[2,10] = interpu(61)
options.iloc[3,10] = interpu(122)

#JPY interp
interpj = scipy.interpolate.interp1d(X, JPY, bounds_error=False, fill_value=scipy.nan)
options.iloc[4,10] = interpj(187) 

#r and rf as decimal
options['r'] = options['r'] / 100
options['rf'] = options['rf'] / 100

#vol for each option
options.iloc[2, 11] = np.std(hist_rates_ret['USD']) * np.sqrt(252)
options.iloc[3, 11] = np.std(hist_rates_ret['USD']) * np.sqrt(252)
options.iloc[0,11] = np.std(hist_rates_ret['EUR']) * np.sqrt(252)
options.iloc[1,11] = np.std(hist_rates_ret['EUR']) * np.sqrt(252)
options.iloc[4,11] = np.std(hist_rates_ret['JPY']) * np.sqrt(252)

#define currency option formula 
# S = spot exchange rate (one unit of foreign currency in AUD)
# K = strike price
# r = domestic rate 
# rf = foreign interest rate
# v = volatility 
# T = ttm 
def currencyoptionprice(callorput, S, K, r, rf, v, T):
    
    d1 = (np.log(S / K) + ((r-rf) + ((v * v * 0.5)) * T) / (v * np.sqrt(T)))
    
    d2 = (np.log(S / K) + ((r - rf) - (v * v * 0.5) * T ) / (v * np.sqrt(T)))
    
             
    if callorput == 'Call':
          
        optionprice = (S * np.exp(-rf * T) * norm.cdf(d1)) - (K * np.exp(-r * T) * norm.cdf(d2))
          
    elif callorput == 'Put':
        
        optionprice = (K * np.exp(-r * T) * norm.cdf(-d2)) - (S * np.exp(-rf * T) * norm.cdf(-d1))
    
    else:
        
        optionprice = 'NA'
    
    return optionprice 

#price options 
optlist = (0, 1, 2, 3, 4)
options['Price']  = ""
for x in optlist:
    ti = options.loc[x, 'Maturity'] /365
    S = 1/ (options.iloc[x, 8])
    K = 1/ options.iloc[x, 7]
    V = options.loc[x, 'vol'] #* np.sqrt(252) annualised above 
    options.iloc[x, 12] = currencyoptionprice( options.iloc[x, 3],
                                              S,
                                              K,
                                              options.loc[x, 'r'],
                                              options.loc[x, 'rf'],
                                              V,
                                              ti                                            
    
)

In [20]:
#define currency option formula 
# S = spot exchange rate (one unit of foreign currency in AUD)
# K = strike price
# r = domestic rate 
# rf = foreign interest rate
# v = volatility 
# T = ttm 
options1 = pd.read_csv('Port3_opt.csv')

optlist = (0, 1, 2, 3, 4)
options1['Price']  = ""
for x in optlist:
    ti = (options1.iloc[x, 4]) /365
    S = (options1.loc[x, 'Spot'])
    K = options1.loc[x, 'Strike']
    V = options1.loc[x, 'vol'] #* np.sqrt(252) annualised above 
    options1.loc[x, 'Price'] = currencyoptionprice( options1.loc[x, 'CallPut'],
                                              S,
                                              K,
                                              options1.loc[x, 'r'],
                                              options1.loc[x, 'rf'],
                                              V,
                                              ti
                                              )
options1

,Curr,CallPut,BoughtSold,Amount,mat,AUD Amount,Strike,Spot,r,rf,vol,Price
0,Euro,Put,S,130,315,196.9697,1.639344,1.626545,0.009140,-0.00420,0.1000,0.0571437
1,Euro,Call,B,140,274,222.2222,1.562500,1.626545,0.009295,-0.00436,0.1000,0.100343
2,USD,Put,B,120,61,157.8947,1.538462,1.467998,0.010138,0.02260,0.1160,0.0726505
3,USD,Call,B,110,122,150.6849,1.428571,1.467998,0.010010,0.02210,0.1160,0.0527516
4,JPY,Call,S,10000,187,115.0880,0.013879,0.013704,0.009978,-0.03583,0.1245,0.000538706


In [24]:
options1['MtM'] = options1['Price'] * options1['Amount']
options1['MtM']

0    7.42868
1     14.048
2    8.71806
3    5.80267
4    5.38706
Name: MtM, dtype: object

In [22]:
#Historical siulation of options

optlist=(0,1,2,3,4)
BS_HS_Sim = pd.DataFrame()
hist = list(range(0, 2760))

for y in hist:
    
    for x in optlist:
        
        ti = options1.loc[x, 'mat'] /365
        S =  (HS_price_opt.iloc[y, x])
        K =  options1.loc[x, 'Strike']
        V = options1.loc[x, 'vol']
        
        BS_HS_Sim.loc[y,x] = currencyoptionprice( options1.loc[x, 'CallPut'],
                                              S,
                                              K,
                                              options1.loc[x, 'r'],
                                              options1.loc[x, 'rf'],
                                              V,
                                              ti
                )
BS_HS_Sim

,0,1,2,3,4
0,0.819535,-0.179580,0.743974,-0.115505,75.392891
1,0.821155,-0.179353,0.732989,-0.116186,76.579095
2,0.816364,-0.180019,0.746950,-0.115308,75.266912
3,0.822869,-0.179111,0.737897,-0.115891,75.893794
4,0.859398,-0.173509,0.766151,-0.113911,71.570187
5,0.834445,-0.177426,0.746832,-0.115316,73.437844
6,0.838330,-0.176842,0.765878,-0.113933,71.646508
7,0.846530,-0.175577,0.763129,-0.114145,72.189098
8,0.835599,-0.177254,0.752717,-0.114911,74.037344
9,0.848357,-0.175290,0.766179,-0.113909,72.106133


In [5]:
MtM = BS_HS_Sim * [130, 140, 120, 110, 10000] * 1000000
MtM[0] = MtM[0].apply(lambda x: x*-1)
MtM[4] = MtM[4].apply(lambda x: x*-1)
PnL = pd.DataFrame(MtM.sum(axis=1) - 17201906.64)
PnL

,0
0,-4.765593e+06
1,-2.609751e+06
2,-5.979485e+06
3,-2.982062e+06
4,2.671603e+06
5,-2.333149e+06
6,-3.941863e+06
7,-6.373965e+05
8,-1.629120e+06
9,-2.768219e+05


In [6]:
MtM

,0,1,2,3,4
0,-9.256106e+06,1.108785e+07,9.297481e+06,5.282981e+06,-3.975896e+06
1,-9.056226e+06,1.137472e+07,1.088754e+07,4.249288e+06,-2.863174e+06
2,-9.647645e+06,1.053118e+07,8.864956e+06,5.571553e+06,-4.097621e+06
3,-8.844910e+06,1.168000e+07,1.017840e+07,4.705075e+06,-3.498721e+06
4,-4.369000e+06,1.865939e+07,6.057463e+06,7.524278e+06,-7.998626e+06
5,-7.420407e+06,1.379271e+07,8.882113e+06,5.560045e+06,-5.945704e+06
6,-6.943527e+06,1.452188e+07,6.097541e+06,7.495408e+06,-7.911254e+06
7,-5.939116e+06,1.609493e+07,6.501367e+06,7.206148e+06,-7.298821e+06
8,-7.278667e+06,1.400827e+07,8.024921e+06,6.141228e+06,-5.322962e+06
9,-5.715779e+06,1.645173e+07,6.053421e+06,7.527192e+06,-7.391483e+06


In [7]:
MtM.sum(axis=1)

0       1.243631e+07
1       1.459216e+07
2       1.122242e+07
3       1.421984e+07
4       1.987351e+07
5       1.486876e+07
6       1.326004e+07
7       1.656451e+07
8       1.557279e+07
9       1.692508e+07
10      1.512276e+07
11      1.717111e+07
12      1.274099e+07
13      1.650266e+07
14      1.557551e+07
15      1.515549e+07
16      1.596219e+07
17      1.920033e+07
18      1.612139e+07
19      1.511921e+07
20      1.445004e+07
21      1.542752e+07
22      1.751796e+07
23      1.650535e+07
24      1.352591e+07
25      1.471303e+07
26      1.626589e+07
27      1.374950e+07
28      1.883874e+07
29      1.552884e+07
            ...     
2730    1.557186e+07
2731    1.750159e+07
2732    1.590788e+07
2733    1.542190e+07
2734    1.587752e+07
2735    1.552889e+07
2736    1.598097e+07
2737    1.551235e+07
2738    1.514210e+07
2739    1.524234e+07
2740    1.623317e+07
2741    1.574311e+07
2742    1.652313e+07
2743    1.648656e+07
2744    1.580989e+07
2745    1.579236e+07
2746    1.515